In [1]:
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00


In [2]:
!pip install pinecone-client
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 12.5 MB/s eta 0:00:00


In [23]:

from langchain.llms import GooglePalm
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import GooglePalmEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import pinecone
import os
import sys

In [6]:
!mkdir pdfs

In [7]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [9]:
print(data)

[Document(page_content=' \n \nP a g e  1 | 22 \n \n \n \n \n \n \nDATA SCIE NCE  \nINTE RVIEW PREPARATION  \n(30 Days o f Interview \nPreparation) \n \n \n#DAY  02  \n  ', metadata={'source': 'pdfs/Day2.pdf', 'page': 0}), Document(page_content=' \n \nP a g e  2 | 22 \n \nQ1. What is Logistic Regression ? \n \nAnswer: \nThe logistic regression technique involves the dependent variable , which can be represent ed in the \nbinary (0 or 1, true or false, yes or no) values, which means that the outcome could only be in either \none form of two. For example, it can be utilized when we need to find the probability of a \nsuccessful or fail event.  \n \n Logistic Regression is used when  the dependent variable ( target) is categorical.  \n \n \nModel  \nOutput = 0 or 1  \nZ = WX + B  \nhΘ(x) = sigmoid (Z)  \nhΘ(x) = log(P(X) / 1 - P(X) ) = WX +B  \n \n  \nIf ‘Z’ goes to infinity, Y(predicted) will become 1 , and if ‘Z’ goes to negative infinity, Y(predicted) \nwill beco me 0.  \nThe output fro

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [11]:
text_chunks = text_splitter.split_documents(data)


In [12]:
len(text_chunks)

60

In [13]:
print(text_chunks[2])

page_content='P a g e  3 | 22 \n \nCost Function  \n \nCost ( hΘ(x) , Y(Actual)) =    -log (hΘ(x)) if y=1 \n                                             -log (1 - hΘ(x)) if y=0 \n \nThis implementation  is for binary  logistic  regression.  For data with more  than 2 classes,  softmax  re\ngression  has to be used.  \n \n \n \nQ2. Difference between logistic and linear regression ? \n \nAnswer: \nLinear and Logistic regression are the most basic form of regression which are commonly used. The \nessential difference between these two is that Logistic regression is used when the dependent variable \nis binary . In contrast, Linear regression is used when the dependent variable is continuous , and the \nnature of the regression  line is linear.  \n \nKey Differences between  Linear and Logistic Regression  \n \nLinear regression models data using continuous numeric value. As against, logistic regression models \nthe data in the binary values.' metadata={'source': 'pdfs/Day2.pdf', 'page': 

In [14]:
os.environ["GOOGLE_API_KEY"] = ""

In [15]:
embedding = GooglePalmEmbeddings()

In [17]:
query_result = embedding.embed_query("Hello World")
print(len(query_result))

768


In [18]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY',"814b0486-9f63-4cc8-aea6-abaefa4b0d7b")
PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV","gcp-starter")

In [22]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name="anantha"

In [24]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embedding=embedding,index_name=index_name)

In [25]:
#docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [26]:
query = "What is Data Science?"

In [27]:
docs = docsearch.similarity_search(query,k=4)

In [28]:
docs

[Document(page_content='Data  Science  \nInterview  Questions  \n(30 days of Interview  Preparation)'),
 Document(page_content='INEURON.AI  \n \n Page 2 \n  \nQ1.  What is the difference between AI,  Data  Science , ML, and DL ? \n \nAns 1 :  \n         \n  \n \nArtificial Intelligence : AI is purely math and scientific exercis e, but when it became computa tional , it \nstarted to solve human problems formalized into a subset of computer science. A rtificial intelligence has \nchanged the original computational statistics paradigm to the modern idea that machines could mimic \nactual human capabilities, such as deci sion making and perfo rming more “human” tasks. Modern AI into \ntwo categories  \n1. General AI - Planning, decision making, identifying objects, recognizing sounds, social & \nbusiness transactions  \n2. Applied AI - driverless/ Autonomous car or machine smartly trade st ocks \n \nMachine Learning : Instead of engineers “teaching” or programming computers to have what th

In [29]:
llm = GooglePalm(temperature=0.7)

In [31]:
QA = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=docsearch.as_retriever())

In [32]:
QA.run(query)

'Data science has many tools, techniques, and algorithms called from these fields, plus others –to handle big data  The goal of data science, somewhat similar to machine learning, is to make accurate predictions and to  automate and perform transactions in real -time, such as purchasing internet traffic or automatically  generating content.'

In [33]:
query1 = "Why we should study Data Science?"


In [34]:
QA.run(query1)

'To make accurate predictions and to automate and perform transactions in real-time'

In [37]:
query2 = "Difference between Machine learning and Data science"

In [38]:
QA.run(query2)

'Data science relies less on math and coding and more on data and building new systems to process the data.\n'

In [39]:
while True:
  user_input = input("Input Prompt:")
  if user_input=='exit':
    print("Exit...")
    sys.exit()
  if user_input=='':
    continue

  result = QA({'query':user_input})
  print("Answer : ",{result['result']})

Input Prompt:Why we should study Data Science?
Answer :  {'To make accurate predictions and to automate and perform transactions in real-time'}
Input Prompt:Difference between Machine learning and Data science
Answer :  {'Machine learning is a subfield of data science.\n\nMachine learning relies less on math and coding and more on data and building new systems to process the \ndata. Relying on the fields of data integration, distributed architecture, aut omated machine learning, data \nvisualization, data engineering, and automated data -driven decisions, data science can cover an entire \nspectrum of data processing, not only the algorithms or statistics related to data.'}
Input Prompt:exit
Exit...


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
